In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Question iii Radial velocity curve:
It is assumed that:
- The system comprises one planet and one star

The following code picks one of the four vectors to proceed. Please click one of the four code slots below to continue:

In [ ]:
# Radial velocity curve plotter input 1

T0 = 0; #Starting time
P = 5; #Period in days
e = 0; #Eccentricity
w = np.pi/2; #Argument of periapsis in rad
i = np.pi/4; #Inclination in rad
m2 = 1.4; #in Jupiter mass
a = 0.05; #in AU
m1 = 1.4; #in Mass of sun
flag = 1;

In [ ]:
# Radial velocity curve plotter input 2

T0 = 5;
P = 6; #in days
e = 0.3;
w = np.pi*3/2;
i = np.pi/8;
m2 = 0.5; #in Jupiter mass
a = 0.2; #in AU
m1 = 0.3; #in Mass of sun
flag = 2;

In [ ]:
# Radial velocity curve plotter input 3

T0 = 10;
P = 10; #in days
e = 0.6;
w = np.pi*3/2;
i = np.pi/3;
m2 = 2.5; #in Jupiter mass
a = 0.01; #in AU
m1 = 1; #in Mass of sun
flag = 3;

In [ ]:
# Radial velocity curve plotter input 4

T0 = 15; 
P = 12; #in days
e = 0.9;
w = np.pi/4;
i = np.pi/4;
m2 = 5; #in Jupiter mass
a = 1; #in AU
m1 = 1.5; #in Mass of sun
flag = 4;

The following are the constants needed for the simulation

In [ ]:
#Constant in SI unit
M_sun = 1.98847e30; #Mass of sun
R_sun = 6.957e8; #Radius of sun
AU = 1.496e11; #Astronomical unit
M_jupiter = 1.89813e27; #Mass of jupiter
R_jupiter = 7.1492e7; #Nominal radius of jupiter
G = 6.67430e-11; #Gracitational constant

#Convert to SI unit
T0 = T0 * 24 * 60 * 60; # s
P = P * 24 * 60 * 60; # s
m2 = m2 * M_jupiter; # kg
m1 = m1 * M_sun; # kg
a = a * AU; # m

The radial velocity can be expressed as the following equation
$$v_r = K_1(cos(\omega + f)+ecos(\omega))$$
where $K_1$ is expressed as:
$$K_1 = \sqrt{\frac{G}{(1-e^2)}}m_2sin(i)(m_1+m_2)^{-1/2}a^{-1/2}$$


In [ ]:
# Get K1
K1 = ((G/(1-(e**2)))**(1/2))*m2*np.sin(i)*((m1 + m2)**(-1/2))*(a**(-1/2)); #in m/s

To get $v_r(t)$, we need to get $f(t)$, where $f(t)$ is defined implicitly by the following series of equations:
$$$$Kepler's equation: $$M = E - esin(E)$$
Definition of mean anomaly: $$M = 2\pi(t - T_0)/P$$
Geometric relationship between eccentric anomlay and true anomaly:$$tan(\frac{f}{2}) = \sqrt{\frac{1+e}{1-e}}tan(\frac{E}{2})$$
Then, the first step is to use time series defined before to yield the series of mean anomaly



In [ ]:
# Get M from t
t = np.linspace(0.0, 20 * 24 * 60 * 60, num=100);
M = 2*np.pi*(t-T0)/P; # in rad
plt.plot(t,M);
plt.title("Mean Anomaly vs time")
plt.xlabel("Time (s)")
plt.ylabel("Mean Anomaly in rad")

One way to find root Kepler's equation from mean anomaly is to use Newton's method. The following code performs the Newton's method algorithm. 
$$E_{n+1} = E_n - \frac{f(E_n)}{f'(E_n)}$$
where function $f(E)$ can be defined as:
$$f = E - esin(E) - M$$


In [ ]:
# Get E from M by using Newton's method initialize
Error_thresold = 0.0000001;
E = np.zeros((np.size(M),));
Error = E - e*np.sin(E) - M;
print(np.linalg.norm(Error));

In [ ]:
# Get E from M by using Newton's method
while np.linalg.norm(Error)>Error_thresold:
    E_old = E;
    function = E_old - e*np.sin(E_old) - M;
    function_derivative = 1 - e*np.cos(E_old);
    E = E_old - function/function_derivative;
    Error = E - e*np.sin(E) - M;

In [ ]:
# Print Newton Method Solver result
print(E);
print(M);
print(np.linalg.norm(Error));
plt.plot(E,M)
plt.title("Eccentric Anomaly vs Mean Anomaly")
plt.xlabel("Eccentric Anomaly in rad")
plt.ylabel("Mean Anomaly in rad")

Then, use geometric equation that relates between eccentric anomaly and true anomaly to yield the series of true anomaly from eccentric anomaly


In [ ]:
# Get f from E
tanf2 = (((1+e)/(1-e))**(1/2))*np.tan(E/2);
f = 2*np.arctan(tanf2); # in rad
plt.plot(t,f);
plt.title("true anomaly vs time")
plt.xlabel("Time (s)")
plt.ylabel("True anomaly in rad")

Finally, plot $v_r$ from the yielded true anomaly from above. The time has converted to days instead of seconds for ease of checking



In [ ]:
# Get Vr vs f
vr = K1 * (np.cos(w + f)+e*np.cos(w)); # in m/s

The following plots the result:

In [ ]:
t = t/(24*60*60); #Convert t to julian days on plot
plt.plot(t,vr);
if flag == 1:
    plt.title("a vector's radial velocity vs time (in julian days)");
elif flag == 2:
    plt.title("b vector's radial velocity vs time (in julian days)");
elif flag == 3:
    plt.title("c vector's radial velocity vs time (in julian days)");
elif flag == 4:
    plt.title("d vector's radial velocity vs time (in julian days)");
plt.xlabel("Julian days")
plt.ylabel("radial velcity (m/s)")